Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [2]:
from itertools import product
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse
from functools import reduce
from copy import copy
from collections import deque

In [3]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [4]:
NUM_POINTS = 100
NUM_SETS = NUM_POINTS
DENSITY = .3

In [5]:
x = make_set_covering_problem(NUM_POINTS, NUM_SETS, .3)
# print("Element at row=42 and column=42:", x[42, 42])

In [6]:
SETS = x.toarray()

In [7]:
sum(sum(SETS))

3066

In [8]:
def fitness(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, -cost

In [28]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, NUM_SETS - 1)
    new_state[index] = not new_state[index]
    return new_state

def tweak2(state):
    new_state = copy(state)
    num_state_to_change = randint(1, sum(state))
    state_to_change = [index for index, value in enumerate(state) if value]

    for i in range(num_state_to_change):
        index = choice(state_to_change)
        state_to_change.remove(index)
        new_state[index] = not new_state[index]

    for _ in range(num_state_to_change-1):
        index = randint(0, NUM_SETS - 1)
        new_state[index] = not new_state[index]
    return new_state

In [31]:
current_state = [choice([False, False, False, False, False, False, False, False, False, False]) for _ in range(NUM_SETS)]
#print("SETS", SETS, current_state, fitness(current_state), sep="\n")
count = 0
for step in range(10000):
    new_state = tweak(current_state)
    count += 2
    if fitness(new_state) >= fitness(current_state):
        current_state = new_state
        print(fitness(current_state))
        count += 1

(19, -1)
(45, -2)
(56, -3)
(72, -4)
(80, -5)
(85, -6)
(89, -7)
(95, -8)
(98, -9)
(100, -10)


In [32]:
current_state = [choice([False, False, False, False, False, False, False, False, False, False]) for _ in range(NUM_SETS)]
#print("SETS", SETS, current_state, fitness(current_state), sep="\n")
count = 0
for step in range(10000):
    #print(step)
    current_fitness = fitness(current_state)
    count += 1
    if current_fitness[0] == NUM_POINTS:
        new_state = tweak2(current_state)
        new_fit = fitness(new_state)
        count += 1
        if new_fit > current_fitness:
            current_state = new_state
            print(fitness(current_state))
            count += 1
    else:
        new_state = tweak(current_state)
        current_state = new_state
    #print("current fitness->",current_fitness)

(100, -18)
(100, -17)
(100, -14)
(100, -13)
(100, -12)
(100, -11)
(100, -10)
(100, -9)
(100, -8)


In [32]:
count

20017